<a href="https://colab.research.google.com/github/vinaysonare/Data-Science-Projects/blob/main/Kaggle_Competition_Titanic(Predict_survival_of_the_passenger).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal**: Predict if the passenger would survive or not

In [52]:
#importing libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [53]:
# Getting dataset from google drive link

trainDataUrl='https://drive.google.com/file/d/1NPbqiqCx_oFGEmRwobXRQhv0U6_EO7IN/view?usp=sharing'
trainfileUrl='https://drive.google.com/uc?id=' + trainDataUrl.split('/')[-2]

#Creating training data Dataframe
train_df  = pd.read_csv(trainfileUrl,encoding='latin-1')
print(train_df.shape)
train_df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
#Creating test data Dataframe
testDataUrl='https://drive.google.com/file/d/12Z_Hz_hSEHjQv12_zWWlr3wOMIjFa3Na/view?usp=sharing'
testfileUrl='https://drive.google.com/uc?id=' + testDataUrl.split('/')[-2]
test_df  = pd.read_csv(testfileUrl,encoding='latin-1')
print(test_df.shape)
test_df.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Finding columns with null data


In [55]:
nullDatas = train_df.isnull().sum()
nullDatas = nullDatas[nullDatas > 0]
nullDatas

Age         177
Cabin       687
Embarked      2
dtype: int64

Here for most of the sample cabin value is not available, hence we can drop this parameter.

In [56]:
train_df= train_df.drop('Cabin',axis = 1)
# Hence removing this column from test data also
test_df=test_df.drop('Cabin',axis = 1)

In [57]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


Replacing null datas. Using mean for age and Fare column and most frequent value in embarked column

In [58]:
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])
nullDatas = train_df.isnull().sum()
nullDatas = nullDatas[nullDatas > 0]
nullDatas

Series([], dtype: int64)

In [59]:
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].mean())
test_df['Embarked'] = test_df['Embarked'].fillna(test_df['Embarked'].mode()[0])
nullDatas = test_df.isnull().sum()
nullDatas = nullDatas[nullDatas > 0]
nullDatas

Series([], dtype: int64)

In [60]:
# As we can strongly say Name of the passenger doesn't affect survival of passenger
train_df= train_df.drop('Name',axis = 1)
test_df= test_df.drop('Name',axis = 1)

Encoding all the string type parameters 

In [61]:
def encodingCat(df,columns):
    df_final=df
    i=0
    for col in columns:
        df1=pd.get_dummies(df[col],drop_first=True)        
        df.drop([col],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:            
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1
    df_final=pd.concat([df,df_final],axis=1)        
    return df_final

In [62]:
#Merging test and train data for encoding
merged = pd.concat([train_df,test_df],axis=0)
merged.shape

(1309, 10)

In [63]:
columns = merged.select_dtypes(include=['object']).columns
encodeMerged = encodingCat(merged,columns)
encodeMerged =encodeMerged.loc[:,~encodeMerged.columns.duplicated()]
encodeMerged.shape

(1309, 938)

In [64]:
#Separating train and test data
trainData = encodeMerged.iloc[:891,:]
testData = encodeMerged.iloc[891:,:]

**Now making prediction using decision tree**

In [65]:
xtrain = trainData.drop('Survived',axis=1)
ytrain = trainData['Survived']
dtModel =  DecisionTreeClassifier(max_depth=5,random_state=0)
dtModel.fit(xtrain,ytrain)


DecisionTreeClassifier(max_depth=5, random_state=0)

In [66]:
xtest = testData.drop('Survived',axis=1)
y_predicted = dtModel.predict(xtest)
predicted_survival = pd.DataFrame(y_predicted)

Writing Above precdiction into csv file

In [67]:
output_df = pd.concat([xtest['PassengerId'],predicted_survival],axis = 1)
output_df.columns = ['PassengerId','Survived']
output_df.to_csv('Titanic Survival prediction.csv', index = False)